In [98]:
import gensim
import os
import collections
import smart_open
import random

import pandas as pd
import numpy as np


from pprint import pprint
import string
translator = str.maketrans('', '', string.punctuation)

# TODO
- Remove punctuations
- Filtering based on word count

In [13]:
def get_train_data():
    """
    SQL here 
    """
    data = pd.read_csv('~/Desktop/grocery_products.csv')
    return data

In [14]:
data = get_train_data()

In [16]:
print(data.shape)

(893626, 17)


In [101]:
selected_data = data[['source_product_id', 'name', 'departments_name', 'categories_name', 'description_en']]
selected_data = selected_data.head(5)

selected_data

,source_product_id,name,departments_name,categories_name,description_en
0,2506104,A08. Golden Nasi Super Combo,Department - (Mchefz Cuisine Menu (mchefz-cuis...,Rice,Served with fried chicken thigh and braised be...
1,2682483,Honey Stars Breakfast Cereal,Food Cupboard,Cereals & Cereal Bars,This product is getting close to its expiry da...
2,2683014,Ice Lemon Tea,Drinks,Tea,Refreshing combination of fruity flavours and ...
3,2768893,Anticavity Maximum Cavity Protection Toothpaste,Health & Beauty,Oral Care,This product is getting close to its expiry da...
4,2771143,Daily Moisture Therapy Dermo-Cleanser Twin Pack,Health & Beauty,Body Care,Daily Moisture Therapy Dermo-Cleanser is a hyp...


Creating document string

In [112]:
selected_data['document'] = selected_data['name'] + ' ' + selected_data['departments_name'] + ' ' + selected_data['categories_name'] + ' ' + selected_data['description_en']

In [113]:
docs = selected_data['document'].tolist()

In [114]:
selected_data[['document']].iloc[2]

document    Ice Lemon Tea Drinks Tea Refreshing combinatio...
Name: 2, dtype: object

In [115]:
pp.pprint(docs)

[ 'A08. Golden Nasi Super Combo Department - (Mchefz Cuisine Menu '
  '(mchefz-cuisine)) Rice Served with fried chicken thigh and braised beef and '
  'cuttlefish, add ons available',
  'Honey Stars Breakfast Cereal Food Cupboard Cereals & Cereal Bars This '
  "product is getting close to its expiry date—don't let this get away from "
  "you! It's still safe for consumption, now enjoy it at a lower "
  'price.<br/><br/>Expiry Date: 2018-07-30<br/><br/>',
  'Ice Lemon Tea Drinks Tea Refreshing combination of fruity flavours and real '
  'tea leaves. Sweet cooling tea taste with a stimulating lemon flavour finish',
  'Anticavity Maximum Cavity Protection Toothpaste Health & Beauty Oral Care '
  "This product is getting close to its expiry date—don't let this get away "
  "from you! It's still safe for use, now enjoy it at a lower price.<br/> "
  '<br/>Expiry Date : 2018-10-13',
  'Daily Moisture Therapy Dermo-Cleanser Twin Pack Health & Beauty Body Care '
  'Daily Moisture Therapy Dermo-

In [118]:
docs = [doc.translate(translator) for doc in docs]

In [119]:
docs[3]

'Anticavity Maximum Cavity Protection Toothpaste Health  Beauty Oral Care This product is getting close to its expiry date—dont let this get away from you Its still safe for use now enjoy it at a lower pricebr brExpiry Date  20181013'

In [120]:
def clean_documents(docs):
    # remove common words and tokenize
    stoplist = set('for a of the and to in'.split())
    docs = [[word for word in document.lower().split() if word not in stoplist]
             for document in docs]

    # remove words that appear only once
    from collections import defaultdict
    frequency = defaultdict(int)
    for text in docs:
        for token in text:
            frequency[token] += 2

    docs = [[token for token in text if frequency[token] > 1] for text in docs]

    pprint(docs)
    return docs

In [121]:
docs = clean_documents(docs)

[['a08',
  'golden',
  'nasi',
  'super',
  'combo',
  'department',
  'mchefz',
  'cuisine',
  'menu',
  'mchefzcuisine',
  'rice',
  'served',
  'with',
  'fried',
  'chicken',
  'thigh',
  'braised',
  'beef',
  'cuttlefish',
  'add',
  'ons',
  'available'],
 ['honey',
  'stars',
  'breakfast',
  'cereal',
  'food',
  'cupboard',
  'cereals',
  'cereal',
  'bars',
  'this',
  'product',
  'is',
  'getting',
  'close',
  'its',
  'expiry',
  'date—dont',
  'let',
  'this',
  'get',
  'away',
  'from',
  'you',
  'its',
  'still',
  'safe',
  'consumption',
  'now',
  'enjoy',
  'it',
  'at',
  'lower',
  'pricebrbrexpiry',
  'date',
  '20180730brbr'],
 ['ice',
  'lemon',
  'tea',
  'drinks',
  'tea',
  'refreshing',
  'combination',
  'fruity',
  'flavours',
  'real',
  'tea',
  'leaves',
  'sweet',
  'cooling',
  'tea',
  'taste',
  'with',
  'stimulating',
  'lemon',
  'flavour',
  'finish'],
 ['anticavity',
  'maximum',
  'cavity',
  'protection',
  'toothpaste',
  'health',
  'b

In [125]:
dictionary = corpora.Dictionary(docs)
dictionary.save(os.path.join('output_data', 'deerwester.dict'))

In [127]:
print(dictionary)

Dictionary(119 unique tokens: ['a08', 'add', 'available', 'beef', 'braised']...)
